# Langserve endpoint 

> A server-side delivery of langchain agents and chains.

In [ ]:
#| default_exp langserve.agent

In [2]:
#| export


import asyncio

import os
from operator import itemgetter

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from langchain.chat_models import ChatOllama

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain.prompts import (ChatPromptTemplate, MessagesPlaceholder,
                               PromptTemplate)
from langchain.schema import Document
from langchain.schema.embeddings import Embeddings
from langchain.schema.language_model import BaseLanguageModel
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.retriever import BaseRetriever
from langchain.schema.runnable import (Runnable, RunnableBranch,
                                       RunnableLambda, RunnableMap)
from langchain.vectorstores import Chroma
from langserve import add_routes
from langchain.pydantic_v1 import BaseModel

from langchain.llms import Ollama
from langchain.memory import ConversationBufferMemory

from langserve import add_routes

from langchain.agents import AgentExecutor
from langchain.tools.render import render_text_description, render_text_description_and_args

from langchain.agents import load_tools
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.agents.output_parsers import ReActSingleInputOutputParser

from alhazen.utils.jats_text_extractor import NxmlDoc
from alhazen.utils.output_parsers import ReActJsonSingleInputOutputParser_llama2

from typing import Dict, List, Optional, Sequence, Union
from uuid import UUID

# Hack to fix Incorrect formatting for Llama Chat models
from alhazen.utils.langchain_utils import ChatPromptValue_to_string
from langchain.prompts.chat import ChatPromptValue

In [3]:
#| export

# Provides access to an agent interface via langserve.
# Very useful for NextJS applications + demos

import nest_asyncio
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.
)
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from alhazen.utils.output_parsers import JsonEnclosedByTextOutputParser

nest_asyncio.apply()

#async_browser = create_async_playwright_browser()
#browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
#tools = browser_toolkit.get_tools()
tools = tools = load_tools([], )

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

chat_model = ChatOllama(model='mixtral')
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
#ChatPromptValue.to_string = ChatPromptValue_to_string

#llm = Ollama(model='llama2:70b')
#llm_with_stop = llm.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# We need to add these input/output schemas because the current AgentExecutor
# is lacking in schemas.
class ChatRequest(BaseModel):
    question: str
    chat_history: Optional[List[Dict[str, str]]]

class Input(BaseModel):
    input: str

class Output(BaseModel):
    output: str

In [4]:
#| export

app = FastAPI(
    title="Alhazen Server",
    version="0.0.1",
    description="An api server using Langchain's Runnable interfaces for Alhazen",
)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"],
)

# Adds routes to the app for using the chain under:
# /invoke
# /batch
# /stream
add_routes(app, agent_executor, input_type=Input, output_type=Output)

In [ ]:
#| export

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8080)